# ポアソン混合分布推定(変分推論)

このシートでは、潜在変数とパラメータを分けることによって、事後分布を近似することを目指す。

$$
p(S, \lambda, \pi | X) \approx q(S)q(\lambda, \pi)
$$

変分推論では、何らかのいい性質を持っている分布の中で、分布pと最も”近い”分布qを探すことを探すことが目的。
特に以下の仮定を置いたものを平均場近似という。
$$
p(z_1, z_2, z_3) \ approx q(z_1)q(z_2)q(z_3)
$$
この下で、各分解された近似分布$q(z_1), q(z_2), q(z_3)$をKLダイバージェンスが最も小さくなるように１つ１つ修正していくのが、平均場近似による変分推論の基本的なアルゴリズム。
例えば、q(z_2)とq(z_3)わかっているとすると、(期待値計算は、<>とかく)
)

$q_{opt.}(z_1) = \text{argmin KL}_{q(z_1)} [q(z_1)q(z_2)q(z_3) || p(z_1, z_2, z_3)]$
$$
\text{KL} [q(z_1)q(z_2)q(z_3) || p(z_1, z_2, z_3)] \\

= - <\ln \frac{p(z_1, z_2, z_3)}{q(z_1)q(z_2)q(z_3)}>_{1,2,3}

$$